In [1]:
import requests

url = "https://api.github.com/repos/huggingface/datasets/issues?page=1&per_page=1"
response = requests.get(url)


In [2]:
response.status_code

200

In [3]:
response.json()

[{'url': 'https://api.github.com/repos/huggingface/datasets/issues/7913',
  'repository_url': 'https://api.github.com/repos/huggingface/datasets',
  'labels_url': 'https://api.github.com/repos/huggingface/datasets/issues/7913/labels{/name}',
  'comments_url': 'https://api.github.com/repos/huggingface/datasets/issues/7913/comments',
  'events_url': 'https://api.github.com/repos/huggingface/datasets/issues/7913/events',
  'html_url': 'https://github.com/huggingface/datasets/pull/7913',
  'id': 3758884376,
  'node_id': 'PR_kwDODunzps66aEsF',
  'number': 7913,
  'title': 'Add lance format support',
  'user': {'login': 'eddyxu',
   'id': 17097,
   'node_id': 'MDQ6VXNlcjE3MDk3',
   'avatar_url': 'https://avatars.githubusercontent.com/u/17097?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/eddyxu',
   'html_url': 'https://github.com/eddyxu',
   'followers_url': 'https://api.github.com/users/eddyxu/followers',
   'following_url': 'https://api.github.com/users/eddyxu/followi

In [ ]:
import os
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")

In [9]:
import time
import math
from pathlib import Path
import pandas as pd
from tqdm.notebook import tqdm


def fetch_issues(
    owner="huggingface",
    repo="datasets",
    num_issues=10_000,
    rate_limit=500_000,
    issues_path=Path("."),
):
    if not issues_path.is_dir():
        issues_path.mkdir(exist_ok=True)

    batch = []
    all_issues = []
    per_page = 100  ## 每页返回的 issue 的数量
    num_pages = math.ceil(num_issues / per_page)
    base_url = "https://api.github.com/repos"

    for page in tqdm(range(num_pages)):
        # 使用 state=all 进行查询来获取 open 和 closed 的issue
        query = f"issues?page={page}&per_page={per_page}&state=all"
        issues = requests.get(f"{base_url}/{owner}/{repo}/{query}", headers=headers)
        batch.extend(issues.json())

        if len(batch) > rate_limit and len(all_issues) < num_issues:
            all_issues.extend(batch)
            batch = []  # 重置batch
            print(f"Reached GitHub rate limit. Sleeping for one hour ...")
            time.sleep(60 * 60 + 1)

    all_issues.extend(batch)
    df = pd.DataFrame.from_records(all_issues)
    
    # 🔥【新增步骤】清洗数据：排除掉那些容易导致类型错误的“脏”列
    # performed_via_github_app 通常是罪魁祸首，因为它有时是 None，有时是字典
    # timeline_url 等也是元数据，对训练模型没用
    cols_to_drop = [
        "performed_via_github_app", 
        "active_lock_reason", 
        "milestone", 
        "repository_url", 
        "events_url", 
        "labels_url"
    ]
    
    # 只删除存在的列（防止报错）
    df = df.drop(columns=[c for c in cols_to_drop if c in df.columns])
    
    # 或者，更安全的做法是：只保留你真正需要的列（推荐 ✅）
    # 比如我们只关心标题、正文、标签、链接和状态
    # meaningful_columns = ["url", "html_url", "id", "number", "title", "user", "labels", "state", "comments", "created_at", "updated_at", "body"]
    # df = df[meaningful_columns]

    # 保存清洗后的数据
    df.to_json(f"{issues_path}/{repo}-issues.jsonl", orient="records", lines=True)
    
    print(
        f"Downloaded all the issues for {repo}! Dataset stored at {issues_path}/{repo}-issues.jsonl"
    )

In [10]:
fetch_issues()


  0%|          | 0/100 [00:00<?, ?it/s]

Downloaded all the issues for datasets! Dataset stored at ./datasets-issues.jsonl


In [13]:
from datasets import load_dataset
issues_dataset = load_dataset("json",data_files = "datasets-issues.jsonl",split="train")
issues_dataset

Dataset({
    features: ['url', 'comments_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'type', 'draft', 'pull_request', 'body', 'closed_by', 'reactions', 'timeline_url', 'state_reason', 'sub_issues_summary', 'issue_dependencies_summary'],
    num_rows: 7831
})

In [14]:
sample = issues_dataset.shuffle(seed=666).select(range(3))

# 打印出 URL 和 pull 请求
for url, pr in zip(sample["html_url"], sample["pull_request"]):
    print(f">> URL: {url}")
    print(f">> Pull request: {pr}\n")

>> URL: https://github.com/huggingface/datasets/pull/6718
>> Pull request: {'url': 'https://api.github.com/repos/huggingface/datasets/pulls/6718', 'html_url': 'https://github.com/huggingface/datasets/pull/6718', 'diff_url': 'https://github.com/huggingface/datasets/pull/6718.diff', 'patch_url': 'https://github.com/huggingface/datasets/pull/6718.patch', 'merged_at': datetime.datetime(2024, 3, 7, 13, 58, 4)}

>> URL: https://github.com/huggingface/datasets/pull/7046
>> Pull request: {'url': 'https://api.github.com/repos/huggingface/datasets/pulls/7046', 'html_url': 'https://github.com/huggingface/datasets/pull/7046', 'diff_url': 'https://github.com/huggingface/datasets/pull/7046.diff', 'patch_url': 'https://github.com/huggingface/datasets/pull/7046.patch', 'merged_at': datetime.datetime(2024, 7, 12, 12, 58, 17)}

>> URL: https://github.com/huggingface/datasets/issues/4291
>> Pull request: None



In [15]:
issues_dataset = issues_dataset.map(
    lambda x: {"is_pull_request": False if x["pull_request"] is None else True}
)

Map:   0%|          | 0/7831 [00:00<?, ? examples/s]

In [18]:
issues_dataset["is_pull_request"]

Column([True, True, False, True, False, ...])

In [19]:
issue_number = 2792
url = f"https://api.github.com/repos/huggingface/datasets/issues/{issue_number}/comments"
response = requests.get(url, headers=headers)
response.json()

[{'url': 'https://api.github.com/repos/huggingface/datasets/issues/comments/897594128',
  'html_url': 'https://github.com/huggingface/datasets/pull/2792#issuecomment-897594128',
  'issue_url': 'https://api.github.com/repos/huggingface/datasets/issues/2792',
  'id': 897594128,
  'node_id': 'IC_kwDODunzps41gDMQ',
  'user': {'login': 'bhavitvyamalik',
   'id': 19718818,
   'node_id': 'MDQ6VXNlcjE5NzE4ODE4',
   'avatar_url': 'https://avatars.githubusercontent.com/u/19718818?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/bhavitvyamalik',
   'html_url': 'https://github.com/bhavitvyamalik',
   'followers_url': 'https://api.github.com/users/bhavitvyamalik/followers',
   'following_url': 'https://api.github.com/users/bhavitvyamalik/following{/other_user}',
   'gists_url': 'https://api.github.com/users/bhavitvyamalik/gists{/gist_id}',
   'starred_url': 'https://api.github.com/users/bhavitvyamalik/starred{/owner}{/repo}',
   'subscriptions_url': 'https://api.github.com/users/

In [26]:
def get_comments(issue_number):
    url = f"https://api.github.com/repos/huggingface/datasets/issues/{issue_number}/comments"
    response = requests.get(url, headers=headers)
    return [r["body"] for r in response.json()]


# 测试我们的函数是否按预期工作
get_comments(2792)

["@albertvillanova my tests are failing here:\r\n```\r\ndataset_name = 'gooaq'\r\n\r\n    def test_load_dataset(self, dataset_name):\r\n        configs = self.dataset_tester.load_all_configs(dataset_name, is_local=True)[:1]\r\n>       self.dataset_tester.check_load_dataset(dataset_name, configs, is_local=True, use_local_dummy_data=True)\r\n\r\ntests/test_dataset_common.py:234: \r\n_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ \r\ntests/test_dataset_common.py:187: in check_load_dataset\r\n    self.parent.assertTrue(len(dataset[split]) > 0)\r\nE   AssertionError: False is not true\r\n```\r\nWhen I try loading dataset on local machine it works fine. Any suggestions on how can I avoid this error?",
 'Thanks for the help, @albertvillanova! All tests are passing now.']

In [29]:
issues_dataset = load_dataset("json",data_files = "datasets-issues.jsonl",split="train")
issues_dataset = issues_dataset.map(
    lambda x: {"is_pull_request": False if x["pull_request"] is None else True}
)
issues_dataset = issues_dataset.select(range(1000))
# 取决于你的网络连接,这可能需要几分钟时间...
issues_with_comments_dataset = issues_dataset.map(
    lambda x: {"comments": get_comments(x["number"])}
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [30]:
from huggingface_hub import notebook_login

notebook_login()

In [32]:
issues_with_comments_dataset.push_to_hub("github-issues")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/yuhuihhh/github-issues/commit/96b951b248a06395d278902c1c12b0593f4e485b', commit_message='Upload dataset', commit_description='', oid='96b951b248a06395d278902c1c12b0593f4e485b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/yuhuihhh/github-issues', endpoint='https://huggingface.co', repo_type='dataset', repo_id='yuhuihhh/github-issues'), pr_revision=None, pr_num=None)

In [33]:
remote_dataset = load_dataset("lewtun/github-issues", split="train")
remote_dataset

README.md: 0.00B [00:00, ?B/s]

c:\Users\62453\miniconda3\envs\hf_env\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\62453\.cache\huggingface\hub\datasets--lewtun--github-issues. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Repo card metadata block was not found. Setting CardData to empty.
Xet Storage is enabled for this repo, but

datasets-issues-with-comments.jsonl:   0%|          | 0.00/12.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3019 [00:00<?, ? examples/s]

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'pull_request', 'body', 'timeline_url', 'performed_via_github_app', 'is_pull_request'],
    num_rows: 3019
})

In [34]:
remote_dataset = load_dataset("yuhuihhh/github-issues", split="train")
remote_dataset

README.md: 0.00B [00:00, ?B/s]

c:\Users\62453\miniconda3\envs\hf_env\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\62453\.cache\huggingface\hub\datasets--yuhuihhh--github-issues. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular H

train-00000-of-00001.parquet:   0%|          | 0.00/1.41M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['url', 'comments_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'type', 'draft', 'pull_request', 'body', 'closed_by', 'reactions', 'timeline_url', 'state_reason', 'sub_issues_summary', 'issue_dependencies_summary', 'is_pull_request'],
    num_rows: 1000
})